## 1. Setup and Configuration

In [ ]:
# Import required libraries
import os
import sys
import yaml
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import pandas as pd
from PIL import Image
import torch
from ultralytics import YOLO

# Add utils to path
sys.path.append('./utils')
from data_utils import (
    load_config, 
    visualize_annotations, 
    count_images_and_labels,
    analyze_dataset_distribution,
    plot_training_results
)

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

print(f"PyTorch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")

In [ ]:

PROJECT_ROOT = Path.cwd()
DATA_DIR = PROJECT_ROOT / 'data' / 'final_train'
MODELS_DIR = PROJECT_ROOT / 'models'
PRETRAINED_DIR = MODELS_DIR / 'pretrained'
TRAINED_DIR = MODELS_DIR / 'trained/combinedV2'
CONFIGS_DIR = PROJECT_ROOT / 'configs'
RESULTS_DIR = PROJECT_ROOT / 'combined/results'
PREDICTIONS_DIR = RESULTS_DIR / 'predictions'
METRICS_DIR = RESULTS_DIR / 'metrics'

for dir_path in [PRETRAINED_DIR, TRAINED_DIR, PREDICTIONS_DIR, METRICS_DIR]:
    dir_path.mkdir(parents=True, exist_ok=True)

print("Project directories initialized")
print(f"Project Root: {PROJECT_ROOT}")
print(f"Data Directory: {DATA_DIR}")
print(f"Models Directory: {MODELS_DIR}")
print(f"Results Directory: {RESULTS_DIR}")

In [ ]:

data_config = load_config(CONFIGS_DIR / 'data_config.yaml')
model_config = load_config(CONFIGS_DIR / 'model_config.yaml')
inference_config = load_config(CONFIGS_DIR / 'inference_config.yaml')

print("Configuration files loaded")
print(f"\n Dataset Config:")
print(f"  - Classes: {data_config['names']}")
print(f"  - Number of classes: {data_config['nc']}")
print(f"  - Image size: {data_config['imgsz']}")

print(f"\n Model Config:")
print(f"  - Model: {model_config['model_name']}")
print(f"  - Epochs: {model_config['epochs']}")
print(f"  - Batch size: {model_config['batch']}")
print(f"  - Optimizer: {model_config['optimizer']}")

## 2. Data Exploration and Validation

In [ ]:
data_exists = DATA_DIR.exists()
print(f"Dataset directory exists: {data_exists}")

if data_exists:
    # Check dataset structure
    required_dirs = [
        DATA_DIR / 'images' / 'train',
        DATA_DIR / 'images' / 'val',
        DATA_DIR / 'labels' / 'train',
        DATA_DIR / 'labels' / 'val'
    ]
    
    print("\n Dataset Structure:")
    for dir_path in required_dirs:
        exists = dir_path.exists()
        status = "" if exists else " MISSING"
        print(f"{status} {dir_path.relative_to(PROJECT_ROOT)}")
else:
    print("\nDataset not found")

In [ ]:
# Count images and labels in each split
if data_exists:
    stats = count_images_and_labels(DATA_DIR)
    
    print("\nDataset Statistics:")
    print("\n" + "="*50)
    for split, counts in stats.items():
        print(f"{split.upper():8s} - Images: {counts['images']:5d}, Labels: {counts['labels']:5d}")
    print("="*50)
    
    total_images = sum(s['images'] for s in stats.values())
    total_labels = sum(s['labels'] for s in stats.values())
    print(f"{'TOTAL':8s} - Images: {total_images:5d}, Labels: {total_labels:5d}")
 
    for split, counts in stats.items():
        if counts['images'] != counts['labels'] and counts['images'] > 0:
            print(f"\n Warning: Mismatch in {split} split - {counts['images']} images but {counts['labels']} labels")
else:
    print(" Skipping - dataset not found")

In [ ]:
if data_exists:
    distribution = analyze_dataset_distribution(DATA_DIR)

    for split, stats in distribution.items():
        if stats['total_images'] > 0:
            print(f"\n{split.upper()}:")
            print(f"  Total Images: {stats['total_images']}")
            print(f"  Total Objects: {stats['total_objects']}")
            print(f"  Avg Objects/Image: {stats['avg_objects_per_image']:.2f}")
            print(f"  Avg Object Size: {stats['avg_object_size']:.4f} (normalized)")
            print(f"  Min Object Size: {stats['min_object_size']:.4f}")
            print(f"  Max Object Size: {stats['max_object_size']:.4f}")

else:
    print("Skipping - dataset not found")

In [ ]:

if data_exists and (DATA_DIR / 'images' / 'train').exists():
    train_images = list((DATA_DIR / 'images' / 'train').glob('*.jpg')) + \
                   list((DATA_DIR / 'images' / 'train').glob('*.png'))
    
    if len(train_images) > 0:
        num_samples = min(6, len(train_images))
        sample_images = np.random.choice(train_images, num_samples, replace=False)
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        for idx, img_path in enumerate(sample_images):
            label_path = DATA_DIR / 'labels' / 'train' / (img_path.stem + '.txt')
            
            # Visualize
            img = visualize_annotations(img_path, label_path, data_config['names'])
            
            axes[idx].imshow(img)
            axes[idx].set_title(f'{img_path.name}', fontsize=10)
            axes[idx].axis('off')
        
        plt.suptitle('Sample Training Images with Annotations', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(RESULTS_DIR / 'sample_annotations.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print(f"\nVisualized {num_samples} sample images")
        print(f"   Saved to: {RESULTS_DIR / 'sample_annotations.png'}")
    else:
        print("No images found in training directory")
else:
    print(" Skipping - dataset not found")

## 3. Model Training

In [ ]:
# Initialize YOLO model
model_name = model_config['model_name']
print(f"Initializing {model_name} model...")

# Load pre-trained model
model = YOLO(model_name)

print(f"Model initialized: {model_name}")
print(f"\nModel Summary:")
print(model.info())

In [ ]:

data_config_path = CONFIGS_DIR / 'data_config.yaml'
training_data_config = data_config.copy()
training_data_config['path'] = str(DATA_DIR.absolute())

training_config_path = CONFIGS_DIR / 'training_data_config.yaml'
with open(training_config_path, 'w') as f:
    yaml.dump(training_data_config, f, default_flow_style=False)

print(f"Training configuration prepared")
print(f"   Config path: {training_config_path}")
print(f"   Dataset path: {training_data_config['path']}")

In [ ]:


results = model.train(
    data=str(training_config_path),
    epochs=40,
    imgsz=model_config['imgsz'],
    batch=model_config['batch'],
    optimizer=model_config['optimizer'],
    lr0=model_config['lr0'],
    lrf=model_config['lrf'],
    momentum=model_config['momentum'],
    weight_decay=model_config['weight_decay'],
    
    hsv_h=model_config['hsv_h'],
    hsv_s=model_config['hsv_s'],
    hsv_v=model_config['hsv_v'],
    degrees=model_config['degrees'],
    translate=model_config['translate'],
    scale=model_config['scale'],
    shear=model_config['shear'],
    perspective=model_config['perspective'],
    flipud=model_config['flipud'],
    fliplr=model_config['fliplr'],
    mosaic=model_config['mosaic'],
    mixup=model_config['mixup'],
    
    box=model_config['box'],
    cls=model_config['cls'],
    dfl=model_config['dfl'],
    patience=10,
    save=model_config['save'],
    save_period=model_config['save_period'],
    plots=model_config['plots'],
    device=model_config['device'],
    amp=model_config['amp'],
    
    # Project paths
    project=str(TRAINED_DIR),
    name='aerial_person_detection',
    exist_ok=True,
)


print("raining Complete!")


In [ ]:

results_path = TRAINED_DIR / 'aerial_person_detection_conminedV2'
best_model_path = results_path / 'weights' / 'best.pt'
last_model_path = results_path / 'weights' / 'last.pt'

print(" Training Results:")
print(f"\n   Results directory: {results_path}")
print(f"   Best model: {best_model_path}")
print(f"   Last model: {last_model_path}")

if best_model_path.exists():
    print(f"\n  Best model saved successfully")
    print(f"     Size: {best_model_path.stat().st_size / (1024*1024):.2f} MB")

## 4. Model Evaluation

In [ ]:
# Load best trained model
if best_model_path.exists():
    trained_model = YOLO(str(best_model_path))
    print(f" Loaded best model from: {best_model_path}")
else:
    print("Best model not found. Please train the model first.")
    trained_model = None

In [ ]:

if trained_model is not None:
    print("\nValidating model on validation set...\n")
    
    val_results = trained_model.val(
        data=str(training_config_path),
        imgsz=model_config['imgsz'],
        batch=model_config['batch'],
        conf=model_config['conf'],
        iou=model_config['iou'],
        device=model_config['device'],
        plots=True,
        save_json=True,
    )
    

    print(f"\n   mAP@0.5: {val_results.box.map50:.4f}")
    print(f"   mAP@0.5:0.95: {val_results.box.map:.4f}")
    print(f"   Precision: {val_results.box.mp:.4f}")
    print(f"   Recall: {val_results.box.mr:.4f}")

else:
    print("Skipping validation - model not loaded")

In [ ]:

if results_path.exists():
    print("Generating training plots...\n")
    
    try:
        plot_training_results(results_path)
        print("Training plots generated")
    except Exception as e:
        print(f"Could not generate custom plots: {e}")
        print("   Check the results directory for YOLO's built-in plots")
else:
    print("Results not found")

In [ ]:
if results_path.exists():
    plot_files = [
        'results.png',
        'confusion_matrix.png',
        'F1_curve.png',
        'PR_curve.png',
        'P_curve.png',
        'R_curve.png'
    ]
    
    available_plots = []
    for plot_file in plot_files:
        plot_path = results_path / plot_file
        if plot_path.exists():
            available_plots.append(plot_path)
    
    if available_plots:
        print(f"\nDisplaying {len(available_plots)} training plots:\n")

        n_plots = len(available_plots)
        n_cols = 2
        n_rows = (n_plots + 1) // 2
        
        fig, axes = plt.subplots(n_rows, n_cols, figsize=(16, 6*n_rows))
        if n_plots == 1:
            axes = [axes]
        else:
            axes = axes.flatten()
        
        for idx, plot_path in enumerate(available_plots):
            img = Image.open(plot_path)
            axes[idx].imshow(img)
            axes[idx].set_title(plot_path.stem.replace('_', ' ').title(), fontsize=12, fontweight='bold')
            axes[idx].axis('off')
        
        for idx in range(len(available_plots), len(axes)):
            axes[idx].axis('off')
        
        plt.tight_layout()
        plt.savefig(METRICS_DIR / 'all_training_plots.png', dpi=150, bbox_inches='tight')
        plt.show()
    else:
        print("No plots found in results directory")
else:
    print("Results directory not found")

## 5. Inference and Visualization


In [ ]:

if trained_model is not None:
    test_images_dir = DATA_DIR / 'images' / 'test'
    
    if test_images_dir.exists():
        test_images = list(test_images_dir.glob('*.jpg')) + list(test_images_dir.glob('*.png'))
        
        if len(test_images) > 0:
            print(f"🔍 Running inference on {len(test_images)} test images...\n")
            
            # Run prediction
            results = trained_model.predict(
                source=str(test_images_dir),
                imgsz=inference_config['imgsz'],
                conf=inference_config['conf'],
                iou=inference_config['iou'],
                max_det=inference_config['max_det'],
                device=inference_config['device'],
                save=inference_config['save_img'],
                save_txt=inference_config['save_txt'],
                save_conf=inference_config['save_conf'],
                show_labels=inference_config['show_labels'],
                show_conf=inference_config['show_conf'],
                show_boxes=inference_config['show_boxes'],
                line_width=inference_config['line_width'],
                project=str(PREDICTIONS_DIR),
                name='test_predictions',
                exist_ok=True,
            )
            
            print(f"\n Inference complete!")
            print(f"   Results saved to: {PREDICTIONS_DIR / 'test_predictions'}")
        else:
            print(" No test images found")
    else:
        print(" Test images directory not found")
        print(f"   Expected at: {test_images_dir}")
else:
    print("  Model not loaded")

In [ ]:
predictions_output = PREDICTIONS_DIR / 'test_predictions'

if predictions_output.exists():
    predicted_images = list(predictions_output.glob('*.jpg')) + list(predictions_output.glob('*.png'))
    
    if len(predicted_images) > 0:
        num_samples = min(6, len(predicted_images))
        sample_predictions = np.random.choice(predicted_images, num_samples, replace=False)
        
        fig, axes = plt.subplots(2, 3, figsize=(18, 12))
        axes = axes.flatten()
        
        for idx, pred_path in enumerate(sample_predictions):
            img = Image.open(pred_path)
            axes[idx].imshow(img)
            axes[idx].set_title(f'{pred_path.name}', fontsize=10)
            axes[idx].axis('off')
        
        plt.suptitle('Sample Predictions on Test Set', fontsize=16, fontweight='bold')
        plt.tight_layout()
        plt.savefig(RESULTS_DIR / 'sample_predictions.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print(f"\nVisualized {num_samples} sample predictions")
        print(f"   Saved to: {RESULTS_DIR / 'sample_predictions.png'}")
    else:
        print(" No prediction images found")
else:
    print("  Predictions directory not found")
    print("   Run inference first")

In [ ]:

if trained_model is not None and test_images_dir.exists():
    test_images = list(test_images_dir.glob('*.jpg')) + list(test_images_dir.glob('*.png'))
    
    if len(test_images) > 0:
        sample_image = np.random.choice(test_images)
        
        print(f" Running inference on: {sample_image.name}\n")
        results = trained_model.predict(
            source=str(sample_image),
            imgsz=inference_config['imgsz'],
            conf=inference_config['conf'],
            save=False,
            verbose=False
        )
        
        result = results[0]
        
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

        original_img = cv2.imread(str(sample_image))
        original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
        ax1.imshow(original_img)
        ax1.set_title('Original Image', fontsize=14, fontweight='bold')
        ax1.axis('off')

        pred_img = result.plot()
        pred_img = cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB)
        ax2.imshow(pred_img)
        
        num_detections = len(result.boxes)
        ax2.set_title(f'Prediction ({num_detections} persons detected)', fontsize=14, fontweight='bold')
        ax2.axis('off')
        
        plt.tight_layout()
        plt.savefig(RESULTS_DIR / 'single_prediction_demo.png', dpi=150, bbox_inches='tight')
        plt.show()
        
        print(f"\nDetection Details:")
        print(f"   Number of detections: {num_detections}")
        
        if num_detections > 0:
            print(f"\n   Confidence scores:")
            for idx, box in enumerate(result.boxes):
                conf = float(box.conf[0])
                print(f"     Detection {idx+1}: {conf:.3f}")
    else:
        print("  No test images found")

## 6. Export and Deployment


In [ ]:

if trained_model is not None:
    print(" Exporting model to different formats...\n")
    
    export_formats = [
        ('onnx', 'ONNX (Open Neural Network Exchange)'),
        ('torchscript', 'TorchScript'),
    ]
    
    exported_models = {}
    
    for format_name, description in export_formats:
        try:
            print(f"Exporting to {description}...")
            export_path = trained_model.export(format=format_name, imgsz=inference_config['imgsz'])
            exported_models[format_name] = export_path
            print(f"  Exported to: {export_path}\n")
        except Exception as e:
            print(f"   Export to {format_name} failed: {e}\n")
    
    if exported_models:
        for format_name, path in exported_models.items():
            print(f"  {format_name.upper()}: {path}")
else:
    print("  Model not loaded")

In [ ]:
if trained_model is not None and best_model_path.exists():
    print("\n" + "="*80)
    print("Model Summary")
    print(f"\n  Model Architecture: {model_name}")
    print(f"  Best Model Path: {best_model_path}")
    print(f"  Model Size: {best_model_path.stat().st_size / (1024*1024):.2f} MB")
    print(f"\n  Training Configuration:")
    print(f"    - Epochs: {model_config['epochs']}")
    print(f"    - Batch Size: {model_config['batch']}")
    print(f"    - Image Size: {model_config['imgsz']}")
    print(f"    - Optimizer: {model_config['optimizer']}")
    print(f"\n  Inference Configuration:")
    print(f"    - Confidence Threshold: {inference_config['conf']}")
    print(f"    - IoU Threshold: {inference_config['iou']}")
    print(f"    - Max Detections: {inference_config['max_det']}")

## 7. Inference on Custom Images/Videos

In [ ]:
def run_custom_inference(source_path, save_dir=None, show_result=True):

    if trained_model is None:
        print("Model not loaded. Please train or load a model first.")
        return
    
    if save_dir is None:
        save_dir = PREDICTIONS_DIR / 'custom'
    
    source_path = Path(source_path)
    
    if not source_path.exists():
        print(f" Source not found: {source_path}")
        return
    
    print(f" Running inference on: {source_path.name}\n")
    
    results = trained_model.predict(
        source=str(source_path),
        imgsz=inference_config['imgsz'],
        conf=inference_config['conf'],
        iou=inference_config['iou'],
        max_det=inference_config['max_det'],
        device=inference_config['device'],
        save=True,
        project=str(save_dir.parent),
        name=save_dir.name,
        exist_ok=True,
    )
    
    print(f" Inference complete!")
    print(f"   Results saved to: {save_dir}")
    
    if show_result and source_path.suffix.lower() in ['.jpg', '.jpeg', '.png']:
        result = results[0]
        num_detections = len(result.boxes)
        
        plt.figure(figsize=(12, 8))
        pred_img = result.plot()
        pred_img = cv2.cvtColor(pred_img, cv2.COLOR_BGR2RGB)
        plt.imshow(pred_img)
        plt.title(f'Detection Result - {num_detections} persons detected', fontsize=14, fontweight='bold')
        plt.axis('off')
        plt.tight_layout()
        plt.show()
        
        print(f"\n   Detections: {num_detections}")
    
    return results


In [ ]:
# # Example: Uncomment and modify to run inference on your custom image
# custom_image_path = '/home/abdallah/Downloads/gettyimages-1151323185-612x612.jpg'
# results = run_custom_inference(custom_image_path)

